# Промышленность

## Описание проекта

Чтобы оптимизировать производственные расходы, металлургического комбината «Стальная птица» требуется уменьшить потребление электроэнергии на этапе обработки стали. Для этого требуется контролировать температуру сплава.

Задача — построить модель, которая будет её предсказывать. Заказчик хочет использовать разработанную модель для имитации технологического процесса.

### Описание процесса обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены на крышке ковша.

Сначала происходит десульфурация — из стали выводят серу и корректируют её химический состав добавлением примесей. Затем сталь легируют — добавляют в неё куски сплава из бункера для сыпучих материалов или порошковую проволоку через специальный трайб-аппарат.

Прежде чем в первый раз ввести легирующие добавки, специалисты производят химический анализ стали и измеряют её температуру. Потом температуру на несколько минут повышают, уже после этого добавляют легирующие материалы и продувают сталь инертным газом, чтобы перемешать, а затем снова проводят измерения. Такой цикл повторяется до тех пор, пока не будут достигнуты нужный химический состав стали и оптимальная температура плавки.

Дальше расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. slab, «плита»).

In [93]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text

In [95]:
# Определим переменные
RANDOM_STATE = 190923
# папка для хранения временных данных
TMP='./cache'

In [96]:
# создаём временный каталог для хранения данных
Path(TMP).mkdir(parents=True, exist_ok=True)

## Подключение к базе данных

In [89]:
db_config = {
    'user': 'praktikum_student',# имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',# пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,# порт подключения
    'db': 'data-science-final'# название базы данных,
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

engine = create_engine(connection_string)
conn = engine.connect()

In [90]:
query = text('''
SELECT * FROM information_schema.tables AS t
WHERE t.table_schema = 'steel';
''')

tables_df = pd.read_sql_query(query, conn)

tables_df

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,data-science-final,steel,data_arc,BASE TABLE,None,None,None,None,None,YES,NO,None
1,data-science-final,steel,data_bulk,BASE TABLE,None,None,None,None,None,YES,NO,None
2,data-science-final,steel,data_bulk_time,BASE TABLE,None,None,None,None,None,YES,NO,None
3,data-science-final,steel,data_gas,BASE TABLE,None,None,None,None,None,YES,NO,None
4,data-science-final,steel,data_temp,BASE TABLE,None,None,None,None,None,YES,NO,None
5,data-science-final,steel,data_wire,BASE TABLE,None,None,None,None,None,YES,NO,None
6,data-science-final,steel,data_wire_time,BASE TABLE,None,None,None,None,None,YES,NO,None


### Описание данных

База данных состоит из нескольких таблиц:
* steel.data_arc — данные об электродах;
* steel.data_bulk — данные об объёме сыпучих материалов;
* steel.data_bulk_time — данные о времени подачи сыпучих материалов;
* steel.data_gas — данные о продувке сплава газом;
* steel.data_temp — данные об измерениях температуры;
* steel.data_wire — данные об объёме проволочных материалов;
* steel.data_wire_time — данные о времени подачи проволочных материалов.

__Таблица steel.data_arc__
* key — номер партии;
* BeginHeat — время начала нагрева;
* EndHeat — время окончания нагрева;
* ActivePower — значение активной мощности;
* ReactivePower — значение реактивной мощности.

__Таблица steel.data_bulk__
* key — номер партии;
* Bulk1 … Bulk15 — объём подаваемого материала.

__Таблица steel.data_bulk_time__
* key — номер партии;
* Bulk1 … Bulk15 — время подачи материала.

__Таблица steel.data_gas__
* key — номер партии;
* gas — объём подаваемого газа.

__Таблица steel.data_temp__
* key — номер партии;
* MesaureTime — время замера;
* Temperature — значение температуры.

__Таблица steel.data_wire__
* key — номер партии;
* Wire1 … Wire15 — объём подаваемых проволочных материалов.

__Таблица steel.data_wire_time__
* key — номер партии;
* Wire1 … Wire15 — время подачи проволочных материалов.

Во всех таблицах столбец `key` содержит номер партии. В таблицах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

### Первичное исследование таблиц

#### Кэширование данных

In [91]:
# создадим вспомогательные функции

def query_fast(query):
    """
    Быстрое выполнение запроса
    
    Параметры:
    ----------
    query: text - запрос
    """
    return pd.read_sql_query(text(query), conn)

def query_info(query, limit=5):
    """
    Получение простой информации по запросу
    
    Параметры:
    ----------
    query: text - запрос
    where: text - условие
    limit: integer - количество записей для вывода
    """
    table_query = text(f'{query} LIMIT {limit};')
    table_df = pd.read_sql_query(table_query, conn)
    display(table_df)
    
    table_count_query = text(f'{query.replace("*", "COUNT(*)")};')
    table_count_df = pd.read_sql_query(table_count_query, conn)
    print(f"Общее количество записей: {table_count_df.loc[0, 'count']}")

In [92]:
# проверим все талбицы на наличие данных и сохраним результаты в локальное хранилище
tables = [
    'steel.data_arc', 
    'steel.data_bulk', 
    'steel.data_bulk_time', 
    'steel.data_gas', 
    'steel.data_temp', 
    'steel.data_wire', 
    'steel.data_wire_time'
]

for table in tables:
    query_info(f'SELECT * FROM {table}')

,key,BeginHeat,EndHeat,ActivePower,ReactivePower
0,1,11:02:14,11:06:02,0.976059,0.687084
1,1,11:07:28,11:10:33,0.805607,0.520285
2,1,11:11:44,11:14:36,0.744363,0.498805
3,1,11:18:14,11:24:19,1.659363,1.062669
4,1,11:26:09,11:28:37,0.692755,0.414397


Общее количество записей: 14876


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,None,None,None,43.0,None,None,None,None,None,None,None,206.0,None,150.0,154.0
1,2,None,None,None,73.0,None,None,None,None,None,None,None,206.0,None,149.0,154.0
2,3,None,None,None,34.0,None,None,None,None,None,None,None,205.0,None,152.0,153.0
3,4,None,None,None,81.0,None,None,None,None,None,None,None,207.0,None,153.0,154.0
4,5,None,None,None,78.0,None,None,None,None,None,None,None,203.0,None,151.0,152.0


Общее количество записей: 3129


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,None,None,None,11:21:30,None,None,None,None,None,None,None,11:03:52,None,11:03:52,11:03:52
1,2,None,None,None,11:46:38,None,None,None,None,None,None,None,11:40:20,None,11:40:20,11:40:20
2,3,None,None,None,12:31:06,None,None,None,None,None,None,None,12:09:40,None,12:09:40,12:09:40
3,4,None,None,None,12:48:43,None,None,None,None,None,None,None,12:41:24,None,12:41:24,12:41:24
4,5,None,None,None,13:18:50,None,None,None,None,None,None,None,13:12:56,None,13:12:56,13:12:56


...

Общее количество записей: 15907


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,60.059998,NaN,None,None,None,None,None,None,None
1,2,96.052315,NaN,None,None,None,None,None,None,None
2,3,91.160157,NaN,None,None,None,None,None,None,None
3,4,89.063515,NaN,None,None,None,None,None,None,None
4,5,89.238236,9.11456,None,None,None,None,None,None,None


Общее количество записей: 3081


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,11:11:41,None,None,None,None,None,None,None,None
1,2,11:46:10,None,None,None,None,None,None,None,None
2,3,12:13:47,None,None,None,None,None,None,None,None
3,4,12:48:05,None,None,None,None,None,None,None,None
4,5,13:18:15,13:32:06,None,None,None,None,None,None,None


Общее количество записей: 3081


Так как количество данных в таблицах не столь велико, то сохраним данные в кэше

In [97]:
# чтобы не мучить базу данных сохраним данные в локальное хранилище
for table in tables:
    query_fast(f'SELECT * FROM {table}').to_csv(TMP + f'/{table}.csv', index=False)

#### Данные об электродах

In [87]:
data_arc = query_fast('''SELECT BeginHeat FROM steel.data_arc''')

data_arc.info()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "beginheat" does not exist
LINE 1: SELECT BeginHeat FROM steel.data_arc
               ^
HINT:  Perhaps you meant to reference the column "data_arc.BeginHeat".

[SQL: SELECT BeginHeat FROM steel.data_arc]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [78]:
def total_seconds(x):
    """
    Вспомогательная функция для подсчёта секунд
    
    Параметры:
    ----------
    x: datetime.time
    
    Результат:
    ----------
    int - общее количество секунд
    """
    return (x.hour * 3600) + (x.minute * 60) + x.second

In [72]:
data_arc['BeginHeatSeconds'] = data_arc['BeginHeat'].head().apply(total_seconds)
data_arc['EndHeatSeconds'] = data_arc['EndHeat'].apply(total_seconds)

# генерация новых признаков
data_arc['deltaHeatSeconds'] = data_arc['EndHeatSeconds'] - data_arc['BeginHeatSeconds']
data_arc['TotalPower'] = data_arc['ActivePower'] + data_arc['ReactivePower']
data_arc['RelPower'] = data_arc['ActivePower'] / data_arc['ReactivePower']


In [73]:
data_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   key               14876 non-null  int64  
 1   BeginHeat         14876 non-null  object 
 2   EndHeat           14876 non-null  object 
 3   ActivePower       14876 non-null  float64
 4   ReactivePower     14876 non-null  float64
 5   BeginHeatSeconds  14876 non-null  int64  
 6   EndHeatSeconds    14876 non-null  int64  
 7   deltaHeatSeconds  14876 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 929.9+ KB


In [74]:
data_arc.head()

,key,BeginHeat,EndHeat,ActivePower,ReactivePower,BeginHeatSeconds,EndHeatSeconds,deltaHeatSeconds
0,1,11:02:14,11:06:02,0.976059,0.687084,39734,39962,-228
1,1,11:07:28,11:10:33,0.805607,0.520285,40048,40233,-185
2,1,11:11:44,11:14:36,0.744363,0.498805,40304,40476,-172
3,1,11:18:14,11:24:19,1.659363,1.062669,40694,41059,-365
4,1,11:26:09,11:28:37,0.692755,0.414397,41169,41317,-148


#### Данные об объёме сыпучих материалов

In [26]:
query_info("SELECT * FROM steel.data_bulk")

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,None,None,None,43.0,None,None,None,None,None,None,None,206.0,None,150.0,154.0
1,2,None,None,None,73.0,None,None,None,None,None,None,None,206.0,None,149.0,154.0
2,3,None,None,None,34.0,None,None,None,None,None,None,None,205.0,None,152.0,153.0
3,4,None,None,None,81.0,None,None,None,None,None,None,None,207.0,None,153.0,154.0
4,5,None,None,None,78.0,None,None,None,None,None,None,None,203.0,None,151.0,152.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      5 non-null      int64  
 1   Bulk 1   0 non-null      object 
 2   Bulk 2   0 non-null      object 
 3   Bulk 3   0 non-null      object 
 4   Bulk 4   5 non-null      float64
 5   Bulk 5   0 non-null      object 
 6   Bulk 6   0 non-null      object 
 7   Bulk 7   0 non-null      object 
 8   Bulk 8   0 non-null      object 
 9   Bulk 9   0 non-null      object 
 10  Bulk 10  0 non-null      object 
 11  Bulk 11  0 non-null      object 
 12  Bulk 12  5 non-null      float64
 13  Bulk 13  0 non-null      object 
 14  Bulk 14  5 non-null      float64
 15  Bulk 15  5 non-null      float64
dtypes: float64(4), int64(1), object(11)
memory usage: 768.0+ bytes

Общее количество записей: 3129


#### Данные о времени подачи сыпучих материалов

In [27]:
query_info("SELECT * FROM steel.data_bulk_time")

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,None,None,None,11:21:30,None,None,None,None,None,None,None,11:03:52,None,11:03:52,11:03:52
1,2,None,None,None,11:46:38,None,None,None,None,None,None,None,11:40:20,None,11:40:20,11:40:20
2,3,None,None,None,12:31:06,None,None,None,None,None,None,None,12:09:40,None,12:09:40,12:09:40
3,4,None,None,None,12:48:43,None,None,None,None,None,None,None,12:41:24,None,12:41:24,12:41:24
4,5,None,None,None,13:18:50,None,None,None,None,None,None,None,13:12:56,None,13:12:56,13:12:56


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      5 non-null      int64 
 1   Bulk 1   0 non-null      object
 2   Bulk 2   0 non-null      object
 3   Bulk 3   0 non-null      object
 4   Bulk 4   5 non-null      object
 5   Bulk 5   0 non-null      object
 6   Bulk 6   0 non-null      object
 7   Bulk 7   0 non-null      object
 8   Bulk 8   0 non-null      object
 9   Bulk 9   0 non-null      object
 10  Bulk 10  0 non-null      object
 11  Bulk 11  0 non-null      object
 12  Bulk 12  5 non-null      object
 13  Bulk 13  0 non-null      object
 14  Bulk 14  5 non-null      object
 15  Bulk 15  5 non-null      object
dtypes: int64(1), object(15)
memory usage: 768.0+ bytes

Общее количество записей: 3129


In [14]:
# закрываем соединение
engine.dispose()